# RAG written purely in Python


In [1]:
# !pip install openai

# Build the vector store

First we build the vector store based on our main document. This can be any document. Here we take the front page of today's NZZ.

In [1]:
the_document_path = "https://nzz.ch" 

In [2]:
from py4ragTools.database import Database

In [3]:
# embedding_model = 'all-minilm'  # 384
embedding_model='nomic-embed-text'  # 768
db = Database(embedding_model=embedding_model)

constructor here


Let us first explore the basic functionality of our method and find out the dimension of the embeddings.

In [4]:
import ollama
text = 'hello world'
embeddings = ollama.embeddings(
            model=embedding_model,
            prompt=text,
        )
print(len(embeddings['embedding']))

768


In [11]:
from py4ragTools.text_tools import TextHelper
th = TextHelper()

title_and_texts = th.load_html(the_document_path)
count=1
for title_text in title_and_texts:
    entry = ". ".join(title_text)  #title_text is a tuple (title, text)
    # Bei Texten der Länge 0 bleibt ollama stehen
    # Sehr kurze Texte sind unbrauchbar für's anschliessende Generieren
    if len(entry) > 100:
        # print(count)
        # print(entry)
        db.add(entry)
        count += 1
print(f'{count} entries in db')  

db.print_db()


69 entries in db
{'Der Schein der starken Frau trügt: Von der Leyens Erfolg hängt von Berlin und Paris ab. Die Kommissionschefin hat einen guten Machtinstinkt. Doch der nützt wenig, solange Berlin und Paris nur mit sich selber beschäftigt sind.': [1.2721706628799438, -0.35706138610839844, -3.2973785400390625, 0.22318334877490997, 0.5589859485626221, 0.7319557666778564, -1.698840856552124, 0.22950011491775513, -0.34304630756378174, 0.9536793231964111, -1.2149187326431274, 0.6574492454528809, 1.897701621055603, -0.040772922337055206, -0.08183364570140839, -0.15522892773151398, -0.26496466994285583, -0.4202285408973694, -1.4861505031585693, 0.6186816096305847, 0.11851498484611511, 0.20494341850280762, -0.07611679285764694, -1.7839546203613281, 1.1864323616027832, 1.0478324890136719, 0.2493322491645813, -0.1440865397453308, -0.6259043216705322, -0.8750982880592346, -0.46937841176986694, -0.6887148022651672, -0.3889336884021759, 0.18625134229660034, -0.22796030342578888, -1.8751293420791626

# Retrieval

We have built our vector store let us retrieve a context for a user-query.

Set up a query regarding today's topics on the newspaper's front page.

In [12]:
user_query = """
Was sagt Kamala Harris?
"""

In [15]:
context = db.query_database(user_query, 0.5)
len(context)


3

In [16]:
context

[(0.5749068751276181,
  'Anwältin der kleinen Leute: Harris verspricht tiefere Steuern für den Mittelstand und keine Immunität für Trump. '),
 (0.5565589366844891,
  'Harris zielt in der ersten TV-Debatte auf Trumps wunde Punkte – er reagiert gereizt und greift die Migranten an: «Sie essen die Hunde». '),
 (0.519328321404761,
  'Der Nostradamus der US-Wahlen sagt einen Sieg für Harris voraus: «Meine Schlüssel sind sehr zuverlässig». ')]

# Augmentation
Now that we have built our "vector database" and retrieved our context, we do the inference part:

First, we augment the prompt using the context we retrieved.

In [17]:
from py4ragTools.prompt_tools import PromptTools

pt = PromptTools()

In [18]:
context_prompt = ""
for (score, text) in context:
    context_prompt += (text + "\\n")
print(context_prompt)

pt.system_prompt(context_prompt)

Anwältin der kleinen Leute: Harris verspricht tiefere Steuern für den Mittelstand und keine Immunität für Trump. \nHarris zielt in der ersten TV-Debatte auf Trumps wunde Punkte – er reagiert gereizt und greift die Migranten an: «Sie essen die Hunde». \nDer Nostradamus der US-Wahlen sagt einen Sieg für Harris voraus: «Meine Schlüssel sind sehr zuverlässig». \n


{'role': 'system',
 'content': ' Context information is below.\n---------------------\nAnwältin der kleinen Leute: Harris verspricht tiefere Steuern für den Mittelstand und keine Immunität für Trump. \\nHarris zielt in der ersten TV-Debatte auf Trumps wunde Punkte – er reagiert gereizt und greift die Migranten an: «Sie essen die Hunde». \\nDer Nostradamus der US-Wahlen sagt einen Sieg für Harris voraus: «Meine Schlüssel sind sehr zuverlässig». \\n\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nIf you do not find the answer in the context information, then say exactly "Ich weiss es nicht" and nothing else. \nIf the context is not related to the question, then say exactly: "Dazu habe ich keine Information" and nothing else:\nAnswer:\n'}

In [19]:
pt.user_prompt(user_query)

{'role': 'user', 'content': ' Query: \nWas sagt Kamala Harris?\n\n'}

In [20]:
list_of_prompts = [
    pt.system_prompt(context_prompt),
    pt.user_prompt(user_query)
]

print(list_of_prompts)

[{'role': 'system', 'content': ' Context information is below.\n---------------------\nAnwältin der kleinen Leute: Harris verspricht tiefere Steuern für den Mittelstand und keine Immunität für Trump. \\nHarris zielt in der ersten TV-Debatte auf Trumps wunde Punkte – er reagiert gereizt und greift die Migranten an: «Sie essen die Hunde». \\nDer Nostradamus der US-Wahlen sagt einen Sieg für Harris voraus: «Meine Schlüssel sind sehr zuverlässig». \\n\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nIf you do not find the answer in the context information, then say exactly "Ich weiss es nicht" and nothing else. \nIf the context is not related to the question, then say exactly: "Dazu habe ich keine Information" and nothing else:\nAnswer:\n'}, {'role': 'user', 'content': ' Query: \nWas sagt Kamala Harris?\n\n'}]


# Generation

Let us generate a response using the augmented prompt.

First, initialize the generation model:

In [21]:
from py4ragTools.generation_tools import GenerationTools
gt = GenerationTools(generation_model='Mistral')

constructor here


In [22]:
response = gt.get_response(list_of_prompts) 
print(response.choices[0].message.content)

 Harris zielt in der ersten TV-Debatte auf Trumps wunde Punkte – sie spricht ihn an. (Translation: Harris aimed at Trump's vulnerable points in the first TV debate – she speaks to him.)
